In [4]:
import re
import nltk
import os
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
import io
import threading
import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk_data_path = "D:/nltk_data"  # Change this to your desired directory
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)

# Append the path to NLTK's data search paths
nltk.data.path.append(nltk_data_path)

# Download the required NLTK data to the custom path
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('omw-1.4', download_dir=nltk_data_path)
from nltk.tokenize import word_tokenize
nltk.data.path.append("D:/nltk_data")

[nltk_data] Downloading package punkt to D:/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to D:/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to D:/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to D:/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
df = pd.read_csv("cleaned_data.csv")
df.head()

,title,categories/keyword,author_tags
0,childhood trauma and childhood mental disorder...,journal of nervous and mental disease,"['childhood mental disorder', 'childhood traum..."
1,morphological plasticity of hyperelongated cel...,applied microbiology and biotechnology,"['cyanobacteria', 'elongation factor p', 'hype..."


In [6]:
def clean_text(text):
    """Remove HTML tags, special characters, and punctuation."""
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and punctuation
    return text

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens): #running -> run / better -> good
    return [lemmatizer.lemmatize(token) for token in tokens]

def tokenize_text(text): # text = "The dogs are running fast." -> Output: ['The', 'dogs', 'are', 'running', 'fast', '.']
    tokens = word_tokenize(text, language='english', preserve_line=True)
    return tokens

def handle_missing_data(text, placeholder='Missing'): # Input = " " -> #Output: "Missing"
    """Handle missing or noisy data."""
    if pd.isnull(text) or text.strip() == "":
        return placeholder
    return text

def to_lowercase(tokens):
    """Convert tokens to lowercase."""
    return [token.lower() for token in tokens]

def remove_stopwords(tokens): # remove a, an, the, this, etc.
    """Remove stopwords."""
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token not in stop_words]

def preprocess_text(text):
    """Complete text preprocessing pipeline."""
    text = handle_missing_data(text)  # Handle missing or noisy data
    text = clean_text(text)          # Clean text (remove unwanted characters)
    tokens = tokenize_text(text)     # Tokenize text
    tokens = to_lowercase(tokens)    # Convert to lowercase
    tokens = remove_stopwords(tokens)  # Remove stopwords
    tokens = lemmatize_tokens(tokens)  # Or stem_tokens(tokens) for stemming
    return ' '.join(tokens)      

In [7]:
kafka_broker = 'localhost:29092'

In [10]:
def consume_messages():
    consumer = KafkaConsumer(
        'processed_data',
        bootstrap_servers=[kafka_broker],
        enable_auto_commit=True,
        value_deserializer=lambda x: x.decode('utf-8')
    )
    output_file_path = 'preprocess_data.csv'
    print('Running Consumer')
    producer = KafkaProducer(
        bootstrap_servers=[kafka_broker],
        linger_ms=5000,
        acks='all',
        max_block_ms=60000
    )

    try:
        for message in consumer:
            print(f"Received message: [{message.timestamp}:{message.offset}] {message.value}")
            
            # Write the received message (CSV content) to a file
            with open(output_file_path, 'w') as f:
                f.write(message.value)  # Save the content to the file

            print(f"Message saved to {output_file_path}")
            
            # Read the CSV file
            df = pd.read_csv(output_file_path)
            
            # Data cleaning steps
            df['title'] = df['title'].apply(preprocess_text)
            df['categories/keyword'] = df['categories/keyword'].apply(preprocess_text)
            df['author_tags'] = df['author_tags'].apply(preprocess_text)
            # Save the cleaned DataFrame
            
            # Save cleaned data to a new CSV
            df.to_csv(output_file_path , index=False)
            producer.send('response_from_3', "Success")  # Send the cleaned CSV content back
            time.sleep(2)  # Simulate some delay

    except Exception as e:
        print(f"Error while consuming: {e}")

# Producer function
# def produce_messages():
#     csv_file_path = 'preprocess_data.csv'
#     producer = KafkaProducer(
#         bootstrap_servers=[kafka_broker],
#         linger_ms=5000,
#         acks='all',
#         max_block_ms=60000
#     )
#     while not os.path.exists(csv_file_path):
#         print(f"Waiting for {csv_file_path} to appear...")
#         time.sleep(5)  # Wait for 5 seconds before checking again

#     print(f"{csv_file_path} found! Proceeding to send data...")
#     with open(csv_file_path, 'r') as csvfile:
#         csv_content = csvfile.read()  # Read entire file content

#     # Send the entire CSV content to Kafka as a single message
#     print(f'Sending entire CSV content to Kafka: {csv_content[:100]}...')  # Display first 100 chars for logging
#     producer.send('preprocess_data', csv_content.encode('utf-8'))  # Send the CSV content
#     time.sleep(2)


    # Ensure all messages are sent before exiting
    producer.flush()

# Running producer and consumer in separate threads
# producer_thread = threading.Thread(target=produce_messages)
consumer_thread = threading.Thread(target=consume_messages)

# producer_thread.start()
consumer_thread.start()

# producer_thread.join()
consumer_thread.join()

Exception in thread Thread-10 (consume_messages):
Traceback (most recent call last):
  File "d:\anaconda3\envs\dsde-cedt\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "d:\anaconda3\envs\dsde-cedt\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "d:\anaconda3\envs\dsde-cedt\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_22720\3434032373.py", line 2, in consume_messages
  File "d:\anaconda3\envs\dsde-cedt\Lib\site-packages\kafka\consumer\group.py", line 356, in __init__
    self._client = KafkaClient(metrics=self._metrics, **self.config)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\envs\dsde-cedt\Lib\site-packages\kafka\client_async.py", line 244, in __init__
    self.config['api_version'] = self.check_version(timeout=check_timeout)
                                 ^^^^^^^^^^^^

In [9]:
df = pd.read_csv('data/2018_cleaned.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/2018_cleaned.csv'

In [ ]:
tfidf_vectorizer_titles = TfidfVectorizer(max_features=10000, min_df=1, max_df=0.8)
df['title_tfidf'] = list(tfidf_vectorizer_titles.fit_transform(df['title']).toarray())

# Sentence Embeddings for abstracts
# df['tags_tfidf'] = list(tfidf_vectorizer.fit_transform(df['author_tags']).toarray())

# Save extracted features
df.to_pickle("processed_features.pkl")
# df['author_tags'].head()
# df[['title_tfidf', 'tags_tfidf']].head()
# Get the feature names (words) from the vectorizer
vocab = tfidf_vectorizer_titles.get_feature_names_out()
title_tfidf_vector = df['title_tfidf'][1]  # TF-IDF vector
nonzero_indices = [i for i, value in enumerate(title_tfidf_vector) if value != 0]

# Print words and their corresponding TF-IDF scores
for index in nonzero_indices:
    print(f"Word: {vocab[index]}, TF-IDF Score: {title_tfidf_vector[index]}")

Word: applying, TF-IDF Score: 0.3403172942934855
Word: caregiver, TF-IDF Score: 0.3179765854455264
Word: case, TF-IDF Score: 0.21998024295224758
Word: distance, TF-IDF Score: 0.3179765854455264
Word: elderlyarticle, TF-IDF Score: 0.35805011251327074
Word: internationalization, TF-IDF Score: 0.35805011251327074
Word: japanese, TF-IDF Score: 0.327735659539065
Word: psychic, TF-IDF Score: 0.35805011251327074
Word: service, TF-IDF Score: 0.27294668374665065
Word: study, TF-IDF Score: 0.18609364123438332
Word: thai, TF-IDF Score: 0.1800932172800236


In [ ]:
# Apply preprocessing to specific columns (e.g., 'title' and 'abstract')
df['title'] = df['title'].apply(preprocess_text)
df['categories/keyword'] = df['categories/keyword'].apply(preprocess_text)
df['author_tags'] = df['author_tags'].apply(preprocess_text)



# Save the cleaned DataFrame
df.to_csv('data/2018_cleaned_processed.csv', index=False)

NameError: name 'df' is not defined

In [ ]:
new_df = pd.read_csv('data/2018_cleaned_processed.csv')
new_df.head()

,title,categories/keyword,author_tags,title_tfidf
0,hydrogeological parameter distribution estimat...,international journal civil engineering techno...,groundwater model hydraulic conductivity krigi...,[0. 0. 0. ... 0. 0. 0.]
1,applying psychic distance service internationa...,journal asiapacific business,cultural distance health care service japan pe...,[0. 0. 0. ... 0. 0. 0.]
2,estimation aloeemodin content cassia grandis c...,indian journal pharmaceutical science,aloeemodin content cassia garrettiana cassia g...,[0. 0. 0. ... 0. 0. 0.]
3,antirice pathogenic microbial activity persica...,science technology asia,antimicrobial activity essential oil persicari...,[0. 0. 0. ... 0. 0. 0.]
4,line orf protein upregulated reactive oxygen s...,cancer genomics proteomics,hne bladder cancer cancer progression immunohi...,[0. 0. 0. ... 0. 0. 0.]


In [ ]:
# Example pipeline for feature extraction
# TF-IDF for titles
tfidf_vectorizer_titles = TfidfVectorizer(max_features=10000, min_df=0.01, max_df=0.8) #exclude > 80% duplicate and < 1%
df['title_tfidf']  = list(tfidf_vectorizer_titles.fit_transform(df['title']).toarray())

X_tfidf = tfidf_vectorizer_titles.fit_transform(df['title']).toarray()

# Get the feature names (the words)
header = tfidf_vectorizer_titles.get_feature_names_out()

# Convert the TF-IDF array to a DataFrame
df_tfidf = pd.DataFrame(X_tfidf, columns=header)

# Now you can add this DataFrame back to your original DataFrame
df = pd.concat([df, df_tfidf], axis=1)
df = df.drop(columns=['title_tfidf'])

df.head()
# sample_tfidf = tfidf_vectorizer_titles.fit_transform()


# # Sentence Embeddings for abstracts
# # df['tags_tfidf'] = list(tfidf_vectorizer.fit_transform(df['author_tags']).toarray())

# # Save extracted features
# df.to_pickle("processed_features.pkl")
# # df['author_tags'].head()
# # df[['title_tfidf', 'tags_tfidf']].head()

# print(name_tfidf)

,title,categories/keyword,author_tags,access,acid,activity,acute,among,analysis,application,...,thailandarticle,thailandarticleopen,therapy,treatment,two,use,using,via,virus,water
0,hydrogeological parameter distribution estimat...,international journal civil engineering techno...,groundwater model hydraulic conductivity krigi...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.634454,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,applying psychic distance service internationa...,journal asiapacific business,cultural distance health care service japan pe...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,estimation aloeemodin content cassia grandis c...,indian journal pharmaceutical science,aloeemodin content cassia garrettiana cassia g...,0.280471,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.540294,0.0,0.0,0.0
3,antirice pathogenic microbial activity persica...,science technology asia,antimicrobial activity essential oil persicari...,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,line orf protein upregulated reactive oxygen s...,cancer genomics proteomics,hne bladder cancer cancer progression immunohi...,0.207590,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [ ]:
tfidf_vectorizer_titles = TfidfVectorizer(max_features=5000, min_df=0.01, max_df=0.8)
title_tfidf = tfidf_vectorizer_titles.fit_transform(df['title'])

# TF-IDF for tags
tfidf_vectorizer_tags = TfidfVectorizer(max_features=5000, min_df=0.01, max_df=0.8)
tags_tfidf = tfidf_vectorizer_tags.fit_transform(df['author_tags'])

# Optionally, get the feature names (terms)
title_feature_names = tfidf_vectorizer_titles.get_feature_names_out()
tags_feature_names = tfidf_vectorizer_tags.get_feature_names_out()

In [ ]:
from scipy.sparse import hstack

# Combine the separate TF-IDF features into one feature matrix
X = hstack([title_tfidf, tags_tfidf])

# Convert the combined features to a dense format (optional, depending on model)
# X = X.toarray()  # Only if your model requires dense arrays, some models like Logistic Regression do
X.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.28047099, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])